In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0303 15:37:24.924027 4590425536 deprecation.py:323] From <ipython-input-1-3c4f46c1a93b>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0303 15:37:24.924958 4590425536 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0303 15:37:24.926431 4590425536 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz


W0303 15:37:25.154123 4590425536 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
import numpy as np
np.set_printoptions(suppress=True) 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
for folder in ['out/', 'model/']:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [4]:
mb_size = 32
Z_dim = 10
X_dim = mnist.train.images.shape[1] # 784
lam = 10
n_disc = 5

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

def discriminator(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    return D_logit # 不使用 sigmoid

def sample_Z(m,n):
    return np.random.uniform(-1., 1., size=[m,n])

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

G_sample = generator(Z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

eps = tf.random_uniform([mb_size, 1], minval=0., maxval=1.)
X_inter = eps*X + (1. - eps)*G_sample
grad = tf.gradients(discriminator(X_inter), [X_inter])[0] # 所有的 weight
grad_norm = tf.sqrt(tf.reduce_sum((grad)**2, axis=1))
grad_pen = lam*tf.reduce_mean((grad_norm - 1)**2)

D_loss = -(tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)) + grad_pen # D_real 與 D_fake 越遠越好
G_loss = -tf.reduce_mean(D_fake)

D_solver = tf.train.AdamOptimizer(learning_rate=0.0001, beta1=0.5).minimize(D_loss, var_list=theta_D) # 使用 Adam
G_solver = tf.train.AdamOptimizer(learning_rate=0.0001, beta1=0.5).minimize(G_loss, var_list=theta_G)

W0303 15:37:25.605237 4590425536 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/slot_creator.py:187: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
# saver.restore(sess, "model/save_net.ckpt")

In [6]:
i=0
for it in range(1000000):   
    for _ in range(n_disc):
        X_mb, _ =  mnist.train.next_batch(mb_size)
        _, D_l = sess.run([D_solver, D_loss], feed_dict={X:X_mb, Z:sample_Z(mb_size, Z_dim)})
        
    _, G_l = sess.run([G_solver, G_loss], feed_dict={Z:sample_Z(mb_size, Z_dim)})
    
    if it%1000==0:
        print('iter:{} [D loss:{:.4}] [G loss:{:.4}]'.format(it, D_l, G_l))
        
    if it%1000==0:
        samples = sess.run(G_sample, feed_dict={Z:sample_Z(16, Z_dim)})
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        i+=1
        
    if it%1000==0:
        save_path = saver.save(sess, "model/save_net.ckpt")

iter:0 [D loss:1.233] [G loss:0.6728]
iter:1000 [D loss:-3.845] [G loss:0.1338]
iter:2000 [D loss:-2.632] [G loss:-0.3145]
iter:3000 [D loss:-2.162] [G loss:-0.4983]
iter:4000 [D loss:-2.145] [G loss:-0.6747]
iter:5000 [D loss:-2.099] [G loss:-0.8692]
iter:6000 [D loss:-1.849] [G loss:-1.015]
iter:7000 [D loss:-1.793] [G loss:-0.9601]
iter:8000 [D loss:-1.71] [G loss:-0.9164]
iter:9000 [D loss:-1.972] [G loss:-0.8961]
iter:10000 [D loss:-1.578] [G loss:-0.9491]
iter:11000 [D loss:-1.707] [G loss:-0.4466]
iter:12000 [D loss:-2.06] [G loss:-0.1195]
iter:13000 [D loss:-1.999] [G loss:-0.4045]
iter:14000 [D loss:-2.161] [G loss:-0.2075]
iter:15000 [D loss:-1.907] [G loss:-0.102]
iter:16000 [D loss:-1.925] [G loss:-0.167]
iter:17000 [D loss:-1.823] [G loss:-0.07738]
iter:18000 [D loss:-1.791] [G loss:0.06274]
iter:19000 [D loss:-1.765] [G loss:0.208]
iter:20000 [D loss:-1.932] [G loss:0.1694]
iter:21000 [D loss:-1.623] [G loss:0.03835]
iter:22000 [D loss:-1.681] [G loss:-0.06566]
iter:23000

KeyboardInterrupt: 